In [ ]:
import xml.etree.ElementTree as ET
import csv

def extract_wikidata_id(url):
    """Extrait le QID Wikidata d'une URL complète"""
    if not url:
        return None
    return url.rsplit("/", 1)[-1]

def extraire_entites_vers_csv(fichier_xml, fichier_csv, element_tag, qid_only=False):
    """s
    Extrait les xml:id et liens Wikidata d'un fichier TEI XML
    et les sauvegarde dans un fichier CSV.

    Args:
        fichier_xml: chemin du fichier XML
        fichier_csv: chemin du fichier CSV de sortie
        element_tag: tag TEI à rechercher ('place', 'person', etc.)
        qid_only: si True, convertit les URLs en QID uniquement
    """
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}
    tree = ET.parse(fichier_xml)
    root = tree.getroot()

    elements = root.findall(f'.//tei:{element_tag}', namespaces)

    with open(fichier_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # Choix de l'en-tête selon qid_only
        header = ['xml_id', 'wikidata_qid'] if qid_only else ['xml_id', 'wikidata_url']
        writer.writerow(header)

        for el in elements:
            xml_id = el.get('{http://www.w3.org/XML/1998/namespace}id')
            source = el.get('source')
            if not xml_id or not source:
                continue

            if qid_only:
                source = extract_wikidata_id(source)

            writer.writerow([xml_id, source])

    print(f"Conversion des XML en csv terminée ! {len(elements)} éléments '{element_tag}' extraits vers {fichier_csv}")

# 🔹 Exemples d'utilisation :

# Lieux (on ne garde que le QID)
extraire_entites_vers_csv("IndexLieux.xml", "lieux.csv", "place", qid_only=True)

# Personnes (URL complète)
extraire_entites_vers_csv("IndexPersonnes.xml", "personnes.csv", "person", qid_only=True)


Conversion terminée ! 431 éléments 'place' extraits vers lieux.csv
Conversion terminée ! 2100 éléments 'person' extraits vers personnes.csv
